In [ ]:
from googleapiclient.discovery import build
import csv
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import scipy
import networkx as nx
from itertools import combinations
import math
import numpy as np
from collections import defaultdict
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
# Canali target da cui raccogliere i video
channels = [
    "Fanpage.it",
    "La Repubblica",
    "FABRIZIO CORONA",
    "Gianluca Spina",
    "Gianmarco Zagato",
    "La7 Attualità",
    "Rai",
    "Bugalalla Crime",
    "DarkSide - Storia Segreta d'Italia"
]

# Intervallo temporale per i video (due settimana prima dalla riapertura del caso fino ai giorni odierni)
begin_date = datetime(2025, 2, 25)
end_date = datetime(2025, 6, 8)

In [ ]:
# Crea struttura Comment
class Comment:
    def __init__(self, id, video_id, content, author, date, likes, reply_to_id=None):
        self.id = id
        self.video_id = video_id
        self.content = content
        self.author = author
        self.date = datetime.strptime(date, "%Y-%m-%d")
        self.likes = int(likes)
        self.reply_to_id = reply_to_id

    def __repr__(self):
        return f"<Comment by {self.author} on {self.date.strftime('%Y-%m-%d')}>"


In [ ]:
from collections import defaultdict

# Carica file JSON unificati
with open("garlasco_comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)

with open("garlasco_videos.json", "r", encoding="utf-8") as f:
    videos = json.load(f)

# Mappa video_id → canale
Channel_of = {v["video_id"]: v["channel"] for v in videos}

# Mappa canale → lista video_id
Videos_of = defaultdict(list)
for v in videos:
    Videos_of[v["channel"]].append(v["video_id"])

# Mappa video_id → data
Date_of_video = {v["video_id"]: v["published_at"] for v in videos}

# Inizializza dizionari ausiliari
Commenters_of_video = defaultdict(set)
Videos_commented_by = defaultdict(set)
Channels_commented_by = defaultdict(set)
Commenters_of_channel = defaultdict(set)
Comments_of_channel = defaultdict(list)
Comments_by_user = defaultdict(list)

# Popola strutture dai commenti
for c in comments:
    video_id = c["video_id"]
    author = c["author"]
    channel = Channel_of.get(video_id, "Unknown")

    comment_obj = Comment(
        id=c["comment_id"],
        video_id=video_id,
        content=c["content"],
        author=author,
        date=c["date"],
        likes=c["likes"],
        reply_to_id=c.get("reply_to_id")
    )

    Commenters_of_video[video_id].add(author)
    Videos_commented_by[author].add(video_id)
    Channels_commented_by[author].add(channel)
    Commenters_of_channel[channel].add(author)
    Comments_of_channel[channel].append(comment_obj)
    Comments_by_user[author].append(comment_obj)

# Debug: stampa sommaria
print(f"Video totali: {len(Channel_of)}")
print(f"Utenti unici: {len(Comments_by_user)}")
print(f"Canali totali: {len(Videos_of)}")


In [ ]:
# Grafo bipartito: utenti e video
import networkx as nx

# Caricamento dati
with open("garlasco_comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)

# Costruzione grafo bipartito: utenti e video
G_user_video = nx.Graph()
user_video_counts = defaultdict(lambda: defaultdict(int))

# Costruzione delle relazioni utente-video
for c in comments:
    user = c["author"].strip()
    video = c["video_id"].strip()
    if user and video:
        user_video_counts[user][video] += 1

# Aggiunta nodi utenti
for user in user_video_counts:
    G_user_video.add_node(user, type="user")

# Aggiunta nodi video
videos = set(v for user_videos in user_video_counts.values() for v in user_videos)
for video in videos:
    G_user_video.add_node(video, type="video")

# Aggiunta archi pesati
for user, video_dict in user_video_counts.items():
    for video, count in video_dict.items():
        G_user_video.add_edge(user, video, weight=count, relation="commented")

# Esportazione file GEXF
nx.write_gexf(G_user_video, "garlasco_user_video_bipartite.gexf")
print(f"Grafo Utente-Video salvato con {G_user_video.number_of_nodes()} nodi e {G_user_video.number_of_edges()} archi.")

In [ ]:
# Grafo utente–utente: proiezione sui commenti condivisi

# Carica i commenti
with open("garlasco_comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)

# Mappa video_id → utenti che hanno commentato
video_to_users = defaultdict(set)

for c in comments:
    video = c["video_id"]
    user = c["author"]
    video_to_users[video].add(user)

# Costruzione grafo utente–utente
user_graph = nx.Graph()

# Per ogni video, crea archi tra tutti i commentatori
for users in video_to_users.values():
    users = list(users)
    for i in range(len(users)):
        for j in range(i + 1, len(users)):
            u1, u2 = users[i], users[j]

            if user_graph.has_edge(u1, u2):
                user_graph[u1][u2]['weight'] += 1
            else:
                user_graph.add_edge(u1, u2, weight=1)

# Aggiungi attributo tipo nodo
for node in user_graph.nodes:
    user_graph.nodes[node]["type"] = "user"

# Esporta in GEXF
nx.write_gexf(user_graph, "garlasco_user_user_projection.gexf")
print(f"Grafo utente–utente salvato. Nodi: {user_graph.number_of_nodes()}, Archi: {user_graph.number_of_edges()}")


In [ ]:
# Grafo bipartito: utenti e canali
with open("garlasco_videos.json", "r", encoding="utf-8") as f:
    videos = json.load(f)

# Mappa video_id → canale
Channel_of = {v["video_id"]: v["channel"] for v in videos}

G_user_channel = nx.Graph()
user_channel_counts = defaultdict(lambda: defaultdict(int))

# Costruzione delle relazioni utente-canale
for c in comments:
    user = c["author"].strip()
    video = c["video_id"].strip()
    channel = Channel_of.get(video, "Unknown").strip()
    if user and channel:
        user_channel_counts[user][channel] += 1

# Nodi utenti
for user in user_channel_counts:
    G_user_channel.add_node(user, type="user")

# Nodi canali
channels = set(c for uc in user_channel_counts.values() for c in uc)
for channel in channels:
    G_user_channel.add_node(channel, type="channel")

# Aggiunta archi
for user, channel_dict in user_channel_counts.items():
    for channel, count in channel_dict.items():
        G_user_channel.add_edge(user, channel, weight=count, relation="commented")

# Esportazione GEXF
nx.write_gexf(G_user_channel, "garlasco_user_channel_bipartite.gexf")
print(f"Grafo Utente-Canale salvato con {G_user_channel.number_of_nodes()} nodi e {G_user_channel.number_of_edges()} archi.")

In [ ]:
# Grafo dei canali: archi pesati per utenti condivisi
from itertools import combinations

# Carica dati
with open("garlasco_comments.json", "r", encoding="utf-8") as f:
    comments = json.load(f)

with open("garlasco_videos.json", "r", encoding="utf-8") as f:
    videos = json.load(f)

# Mappa video → canale
Channel_of = {v["video_id"]: v["channel"] for v in videos}

# Mappa canale → utenti unici
Commenters_of_channel = defaultdict(set)
for c in comments:
    video_id = c["video_id"]
    author = c["author"]
    channel = Channel_of.get(video_id, "Unknown")
    Commenters_of_channel[channel].add(author)

# Grafo dei canali: archi pesati per utenti condivisi
G_channel = nx.Graph()
channels = list(Commenters_of_channel.keys())

# Aggiungi nodi
for ch in channels:
    G_channel.add_node(ch, type="channel", commenters=len(Commenters_of_channel[ch]))

# Aggiungi archi tra canali con utenti in comune
for ch1, ch2 in combinations(channels, 2):
    shared_users = Commenters_of_channel[ch1] & Commenters_of_channel[ch2]
    if shared_users:
        G_channel.add_edge(ch1, ch2, weight=len(shared_users), relation="shared_commenters")

# Esporta grafo per Gephi
nx.write_gexf(G_channel, "garlasco_channel_channel_projection.gexf")
print(f"Grafo Canale-Canale salvato con {G_channel.number_of_nodes()} nodi e {G_channel.number_of_edges()} archi.")
